In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

Each epoch contains two parts:
- Train loop: try to converge to optimal params
- Validation/Test loop: iterate over test to check if performance is improving

## Loss Function

In [4]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer

Three steps:
- `optimizer.zero_grad()` resets the gradients
- Backpropogate prediction loss with `loss.backward()`
- Call `optimizer.step()` to adjust parameters

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Full Implementation

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # set model to training model; important for batch norm and dropout layers
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropogation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    # set model to eval mode; important for batch norm and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    # Eval model with torch.no_grad()
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313704 [   64/60000]
loss: 2.296324 [ 6464/60000]
loss: 2.282989 [12864/60000]
loss: 2.278295 [19264/60000]
loss: 2.252541 [25664/60000]
loss: 2.236559 [32064/60000]
loss: 2.240596 [38464/60000]
loss: 2.211472 [44864/60000]
loss: 2.216997 [51264/60000]
loss: 2.176825 [57664/60000]
Test Error: 
 Accuracy: 40.8%, Avg loss: 2.176779 

Epoch 2
-------------------------------
loss: 2.192378 [   64/60000]
loss: 2.182785 [ 6464/60000]
loss: 2.131060 [12864/60000]
loss: 2.146219 [19264/60000]
loss: 2.100100 [25664/60000]
loss: 2.045427 [32064/60000]
loss: 2.078462 [38464/60000]
loss: 2.002529 [44864/60000]
loss: 2.014162 [51264/60000]
loss: 1.942229 [57664/60000]
Test Error: 
 Accuracy: 50.1%, Avg loss: 1.940170 

Epoch 3
-------------------------------
loss: 1.975798 [   64/60000]
loss: 1.949045 [ 6464/60000]
loss: 1.835429 [12864/60000]
loss: 1.873831 [19264/60000]
loss: 1.770865 [25664/60000]
loss: 1.716152 [32064/60000]
loss: 1.742766 [38464/